In [1]:
import pandas as pd
import numpy as np

In [2]:
path = '../Data/'
fileName = 'yellow_tripdata_2016-06'
month = int(fileName[-2:])
year = int(fileName[-7:-3])
print(year,month)

2016 6


In [3]:
data = pd.read_csv(path+fileName+'.csv',parse_dates=['tpep_pickup_datetime','tpep_dropoff_datetime'])
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.0,0.3,27.30
2,2,2016-06-09 21:06:36,2016-06-09 21:13:10,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.0,0.3,28.30
4,2,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.0,0.3,17.76


 Удалим поездки с нулевой длительностью, нулевым количеством пассажиров и нулевой дистанцией поездки:

In [4]:
data.drop(data[data.tpep_pickup_datetime == data.tpep_dropoff_datetime].index, inplace=True)
data.drop(data[data.passenger_count == 0].index,inplace=True)
data.drop(data[data.trip_distance == 0.].index,inplace=True)

Теперь удалим поездки, координаты начала которых не относятс к Нью-Йорку:

In [5]:
data.drop(data[data.pickup_latitude < 40.49612].index,inplace=True)
data.drop(data[data.pickup_latitude > 40.91553].index,inplace=True)
data.drop(data[data.pickup_longitude < -74.25559].index,inplace=True)
data.drop(data[data.pickup_longitude > -73.70001].index,inplace=True)

In [6]:
data.reset_index(inplace=True)

In [7]:
cityLatitude = (40.49612,40.91553)
cityLongitude = (-74.25559,-73.70001)

Соотнесём координаты всех строк с полученными номерами районов:

In [8]:
regWidth = -74.2444784 - -74.25559
regHeight = 40.5045082 - 40.49612
# Значения взяты из координат первого региона файла Data/regions.csv, полученного на этапе ознакомления с данными.

In [9]:
def getRegionNumber(longitude, latitude):
    i = ((longitude - cityLongitude[0]) / regWidth).astype(int)
    j = ((latitude - cityLatitude[0]) / regHeight).astype(int)    
    return 50 * i + j + 1

In [10]:
data['region'] = getRegionNumber(data['pickup_longitude'],data['pickup_latitude'])

Агрегируем данные (в том числе, по часам):

In [11]:
from datetime import datetime as dt, date
import calendar

In [12]:
data['tpep_pickup_date'] = data['tpep_pickup_datetime'].dt.date
data['tpep_pickup_hour'] = data['tpep_pickup_datetime'].dt.hour

In [13]:
grouped = data.groupby(['tpep_pickup_date', 'tpep_pickup_hour', 'region'], sort=True).size().reset_index(name='count')
grouped.head()

,tpep_pickup_date,tpep_pickup_hour,region,count
0,2016-06-01,0,1016,1
1,2016-06-01,0,1017,1
2,2016-06-01,0,1031,1
3,2016-06-01,0,1072,1
4,2016-06-01,0,1075,26


GroupBy не учитывает "нулевые пересечения". Их придётся добавить другим способом:
1) Вычислим id объектов, которые оказались в массиве grouped.
    1.1)Для каждой пары "дата-час" из массива grouped найдём число поездок. Пара "дата-час", по сути, служит идентификатором.
    1.2)Определим, скольких элементов для данной пары "дата-час" не хватает в массиве grouped.
2)Сформируем массивы дат, часов и числа поездок на основе полученных данных, и добавим их к grouped.
    

In [14]:
daysInMonth = calendar.monthrange(year, month)[1]
for day in range(1, daysInMonth + 1):
    
    currentDate = date(year, month, day)

    for hour in range(24):
        
        # п.1.1
        region_used_ids = grouped[
            (grouped['tpep_pickup_date'] == currentDate) & 
            (grouped['tpep_pickup_hour'] == hour)
        ].region.values
        
        # 1.2
        region_notused_ids = np.array(tuple(set(range(2500)) - set(region_used_ids)))
        missedCount = region_notused_ids.shape[0]
        
        # 2
        dates = np.array([currentDate]*missedCount)
        hours = np.ones(missedCount, dtype=np.int64)*hour
        cnts = np.zeros(missedCount, dtype=np.int64)
        rows = np.hstack([
            dates.reshape(-1, 1), 
            hours.reshape(-1, 1), 
            region_notused_ids.reshape(-1, 1), 
            cnts.reshape(-1, 1)
        ])                        
        df = pd.DataFrame(rows, columns=grouped.columns)
        grouped = grouped.append(df, ignore_index=True)

Загрузим список номеров самых популярных регионов и выберем те записи из grouped, которые относятся к этим регионам

In [15]:
popularRegions = pd.read_csv('../Data/popularRegions.csv',sep=';',header=None)
popularRegions.head()

,0
0,1074
1,1075
2,1076
3,1124
4,1125


In [16]:
popularIndex = grouped.region.isin(popularRegions.values)
popularTrips = grouped.loc[popularIndex]
popularTrips.head()

,tpep_pickup_date,tpep_pickup_hour,region,count
4,2016-06-01,0,1075,26
5,2016-06-01,0,1076,30
8,2016-06-01,0,1125,39
9,2016-06-01,0,1126,71
10,2016-06-01,0,1127,163


Сделаем timeLine индексом, предварительно объединив столбцы date и hour:

In [17]:
timeLine = pd.to_datetime(popularTrips.tpep_pickup_date) + popularTrips.tpep_pickup_hour.astype('timedelta64[h]')
del popularTrips['tpep_pickup_date']
del popularTrips['tpep_pickup_hour']
popularTrips['timeLine'] = timeLine
cols = popularTrips.columns.tolist()
cols = cols[-1:] + cols[:-1]
popularTrips = popularTrips[cols]
popularTrips.index = popularTrips['timeLine']
popularTrips
del popularTrips['timeLine']
popularTrips.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,region,count
timeLine,,
2016-06-01,1075,26
2016-06-01,1076,30
2016-06-01,1125,39
2016-06-01,1126,71
2016-06-01,1127,163


Наконец, составим dataframe вида:
    По строкам отложена timeLine. По столбцам - номера регионов.
    В ячейке i,j записано число поездок в час i из региона j.

In [18]:
countXregion = pd.DataFrame(index=timeLine)
for region in np.unique(popularTrips['region'].values):
    countXregion[region] = popularTrips[popularTrips['region'] == region]['count']
countXregion.drop_duplicates(inplace=True)
countXregion.head()

,1074,1075,1076,1124,1125,1126,1127,1128,1129,1130,...,1629,1683,1732,1733,1782,2067,2068,2117,2118,2167
2016-06-01 00:00:00,0,26,30,0,39,71,163,181,219,326,...,0,0,0,6,0,0,114,0,169,0
2016-06-01 01:00:00,0,14,21,0,26,49,101,136,144,252,...,0,0,0,0,0,0,27,0,105,0
2016-06-01 02:00:00,0,5,25,0,18,25,42,74,123,245,...,0,0,0,1,0,0,11,0,31,0
2016-06-01 03:00:00,0,2,5,0,4,16,36,36,85,241,...,1,0,0,1,0,0,13,0,51,0
2016-06-01 04:00:00,0,1,8,0,6,26,27,46,54,129,...,0,1,2,0,1,0,23,0,33,0


In [19]:
countXregion.to_csv('../Data/trips/countXregion/countXregion_'+str(year)+'_'+str(month)+'.csv',sep=';')